In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# Silence SettingWithCopyWarning for categorical assignments
pd.options.mode.chained_assignment = None 

# --- Define File Names (Assuming these files are in the same directory) ---
FILE_DEMO = 'df_final_demo.txt'
FILE_WEB_1 = 'df_final_web_data_pt_1.txt'
FILE_WEB_2 = 'df_final_web_data_pt_2.txt'
FILE_EXPERIMENT = 'df_final_experiment_clients.txt'

# --- 1. Data Loading and Merging (Task 1: Data Preparation) ---
def load_and_merge_data(demo_file, web_1_file, web_2_file, experiment_file):
    """Loads and merges all required datasets."""
    print("--- TASK 1: DATA LOADING AND MERGING ---")
    try:
        # Load datasets (using read_csv because your .txt files are CSV-formatted)
        df_demo = pd.read_csv(demo_file)
        df_web_1 = pd.read_csv(web_1_file)
        df_web_2 = pd.read_csv(web_2_file)
        df_experiment = pd.read_csv(experiment_file)

        df_web = pd.concat([df_web_1, df_web_2], ignore_index=True)

        # Standardize client ID column name
        for df_ in [df_experiment, df_demo, df_web]:
            if 'client_id' in df_.columns:
                df_.rename(columns={'client_id': 'Client_ID'}, inplace=True)
        
        # Merge all data
        df_merged = df_experiment.merge(df_demo, on='Client_ID', how='left')
        df_merged = df_merged.merge(df_web, on='Client_ID', how='left')
        print("All datasets merged successfully.")
        return df_merged
    except Exception as e:
        print(f"Error during loading/merging: {e}")
        return None

# --- 1. EDA & Data Cleaning (Task 1: Data Preparation) ---
def perform_eda_and_cleaning(df):
    """Performs initial EDA and cleaning steps."""
    if df is None: return None
    
    print("\n--- TASK 1: EDA AND DATA CLEANING ---")

    # Drop duplicates
    initial_rows = len(df)
    df.drop_duplicates(subset=['Client_ID', 'date_time', 'process_step'], keep='first', inplace=True)

    # Convert date_time and create Date column
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['Date'] = df['date_time'].dt.date
    df['Date'] = pd.to_datetime(df['Date'])

    # Create 'TestGroup' column
    df['TestGroup'] = np.where(df['Variation'] == 'Control', 'Control', 'Test')
    
    print(f"Total rows before cleaning: {initial_rows}")
    print(f"Total rows after cleaning: {len(df)}")
    print(f"Total unique clients after cleaning: {df['Client_ID'].nunique()}")
    
    return df


# --- 2. Client Behavior Analysis (Task 2: Analysis) ---
def analyze_client_behavior(df):
    """Answers questions about client demographics and overall funnel performance."""
    if df is None: return None
    
    print("\n--- TASK 2: CLIENT BEHAVIOR ANALYSIS ---")
    
    # Filter clients with complete demographic data and create a copy
    df_unique_clients = df.drop_duplicates(subset=['Client_ID'])
    df_analysis_clients = df_unique_clients[df_unique_clients['clnt_age'].notnull()].copy() 
    total_clients = df_analysis_clients['Client_ID'].nunique()

    # --- 3.1.1 Gender Analysis (Uses 'gendr') ---
    gender_analysis = df_analysis_clients.groupby('gendr')['Client_ID'].count().sort_values(ascending=False)
    gender_pct = (gender_analysis / total_clients * 100).round(2) 
    print("\n3.1.1 Primary Clients by Gender:")
    print(gender_pct.to_frame(name='Client Pct (%)').to_string()) 

    # --- 3.1.2 Age Group Analysis (Uses 'clnt_age') ---
    age_bins = [18, 30, 45, 60, 100]
    age_labels = ['18-29 (Young)', '30-44 (Middle)', '45-59 (Senior)', '60+ (Retired)']
    df_analysis_clients['Age_Group'] = pd.cut(df_analysis_clients['clnt_age'], bins=age_bins, labels=age_labels, right=False, include_lowest=True)
    
    # Corrected for FutureWarning: Added observed=True
    age_analysis = df_analysis_clients.groupby('Age_Group', observed=True)['Client_ID'].count()
    age_pct = (age_analysis / total_clients * 100).round(2)
    print("\n3.1.2 Primary Clients by Age Group:")
    print(age_pct.to_frame(name='Client Pct (%)').to_string())

    # --- 3.2.1 Tenure Group Analysis (Uses 'clnt_tenure_yr') ---
    tenure_bins = [0, 5, 10, 20, 50]
    tenure_labels = ['<5 (New)', '5-9 (Mid-Term)', '10-19 (Long-Term)', '20+ (Veteran)']
    df_analysis_clients['Tenure_Group'] = pd.cut(df_analysis_clients['clnt_tenure_yr'], bins=tenure_bins, labels=tenure_labels, right=False, include_lowest=True)
    
    # Corrected for FutureWarning: Added observed=True
    tenure_analysis = df_analysis_clients.groupby('Tenure_Group', observed=True)['Client_ID'].count()
    
    # Corrected percentage calculation 
    tenure_pct = tenure_analysis.div(total_clients).mul(100).round(2)
    
    print("\n3.2.1 Primary Clients by Tenure Group (Years Active):")
    print(tenure_pct.to_frame(name='Client Pct (%)').to_string())
    
    # --- 3.3.1 Overall Funnel Performance (Conversion Rate) ---
    print("\n3.3.1 Overall Process Funnel Analysis:")
    
    process_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
    step_activity = {step: df[df['process_step'] == step]['Client_ID'].nunique() for step in process_steps}
    step_activity_series = pd.Series(step_activity).sort_index()

    start_clients = step_activity.get('start', 0)
    confirm_clients = step_activity.get('confirm', 0)
    
    # CRITICAL FIX for AttributeError/NameError: Use the built-in round() function and proper structure
    if start_clients > 0:
        completion_rate = round((confirm_clients / start_clients * 100), 2)
    else:
        completion_rate = 0

    print("Clients reaching each step of the process:")
    print(step_activity_series.to_frame(name='Unique Clients').to_string())
    print(f"\nOverall Process Completion Rate (Confirm/Start): {completion_rate}%")
    


# --- Main Execution Block ---
if __name__ == '__main__':
    df_merged = load_and_merge_data(FILE_DEMO, FILE_WEB_1, FILE_WEB_2, FILE_EXPERIMENT)
    
if df_merged is not None:
    df_cleaned = perform_eda_and_cleaning(df_merged)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# Silence SettingWithCopyWarning for categorical assignments
pd.options.mode.chained_assignment = None 

# --- Define File Names (Assuming these files are in the same directory) ---
FILE_DEMO = 'df_final_demo.txt'
FILE_WEB_1 = 'df_final_web_data_pt_1.txt'
FILE_WEB_2 = 'df_final_web_data_pt_2.txt'
FILE_EXPERIMENT = 'df_final_experiment_clients.txt'

# --- 1. Data Loading and Merging (Task 1: Data Preparation) ---
def load_and_merge_data(demo_file, web_1_file, web_2_file, experiment_file):
    """Loads and merges all required datasets."""
    print("--- TASK 1: DATA LOADING AND MERGING ---")
    try:
        # Load datasets (using read_csv because your .txt files are CSV-formatted)
        df_demo = pd.read_csv(demo_file)
        df_web_1 = pd.read_csv(web_1_file)
        df_web_2 = pd.read_csv(web_2_file)
        df_experiment = pd.read_csv(experiment_file)

        df_web = pd.concat([df_web_1, df_web_2], ignore_index=True)

        # Standardize client ID column name
        for df_ in [df_experiment, df_demo, df_web]:
            if 'client_id' in df_.columns:
                df_.rename(columns={'client_id': 'Client_ID'}, inplace=True)
        
        # Merge all data
        df_merged = df_experiment.merge(df_demo, on='Client_ID', how='left')
        df_merged = df_merged.merge(df_web, on='Client_ID', how='left')
        print("All datasets merged successfully.")
        return df_merged
    except Exception as e:
        print(f"Error during loading/merging: {e}")
        return None

# --- 1. EDA & Data Cleaning (Task 1: Data Preparation) ---
def perform_eda_and_cleaning(df):
    """Performs initial EDA and cleaning steps."""
    if df is None: return None
    
    print("\n--- TASK 1: EDA AND DATA CLEANING ---")

    # Drop duplicates
    initial_rows = len(df)
    df.drop_duplicates(subset=['Client_ID', 'date_time', 'process_step'], keep='first', inplace=True)

    # Convert date_time and create Date column
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['Date'] = df['date_time'].dt.date
    df['Date'] = pd.to_datetime(df['Date'])

    # Create 'TestGroup' column
    df['TestGroup'] = np.where(df['Variation'] == 'Control', 'Control', 'Test')
    
    print(f"Total rows before cleaning: {initial_rows}")
    print(f"Total rows after cleaning: {len(df)}")
    print(f"Total unique clients after cleaning: {df['Client_ID'].nunique()}")
    
    return df


# --- 2. Client Behavior Analysis (Task 2: Analysis) ---
def analyze_client_behavior(df):
    """Answers questions about client demographics and overall funnel performance."""
    if df is None: return None
    
    print("\n--- TASK 2: CLIENT BEHAVIOR ANALYSIS ---")
    
    # Filter clients with complete demographic data and create a copy
    df_unique_clients = df.drop_duplicates(subset=['Client_ID'])
    df_analysis_clients = df_unique_clients[df_unique_clients['clnt_age'].notnull()].copy() 
    total_clients = df_analysis_clients['Client_ID'].nunique()

    # --- 3.1.1 Gender Analysis (Uses 'gendr') ---
    gender_analysis = df_analysis_clients.groupby('gendr')['Client_ID'].count().sort_values(ascending=False)
    gender_pct = (gender_analysis / total_clients * 100).round(2) 
    print("\n3.1.1 Primary Clients by Gender:")
    print(gender_pct.to_frame(name='Client Pct (%)').to_string()) 

    # --- 3.1.2 Age Group Analysis (Uses 'clnt_age') ---
    age_bins = [18, 30, 45, 60, 100]
    age_labels = ['18-29 (Young)', '30-44 (Middle)', '45-59 (Senior)', '60+ (Retired)']
    df_analysis_clients['Age_Group'] = pd.cut(df_analysis_clients['clnt_age'], bins=age_bins, labels=age_labels, right=False, include_lowest=True)
    
    # Corrected for FutureWarning: Added observed=True
    age_analysis = df_analysis_clients.groupby('Age_Group', observed=True)['Client_ID'].count()
    age_pct = (age_analysis / total_clients * 100).round(2)
    print("\n3.1.2 Primary Clients by Age Group:")
    print(age_pct.to_frame(name='Client Pct (%)').to_string())

    # --- 3.2.1 Tenure Group Analysis (Uses 'clnt_tenure_yr') ---
    tenure_bins = [0, 5, 10, 20, 50]
    tenure_labels = ['<5 (New)', '5-9 (Mid-Term)', '10-19 (Long-Term)', '20+ (Veteran)']
    df_analysis_clients['Tenure_Group'] = pd.cut(df_analysis_clients['clnt_tenure_yr'], bins=tenure_bins, labels=tenure_labels, right=False, include_lowest=True)
    
    # Corrected for FutureWarning: Added observed=True
    tenure_analysis = df_analysis_clients.groupby('Tenure_Group', observed=True)['Client_ID'].count()
    
    # Corrected percentage calculation 
    tenure_pct = tenure_analysis.div(total_clients).mul(100).round(2)
    
    print("\n3.2.1 Primary Clients by Tenure Group (Years Active):")
    print(tenure_pct.to_frame(name='Client Pct (%)').to_string())
    
    # --- 3.3.1 Overall Funnel Performance (Conversion Rate) ---
    print("\n3.3.1 Overall Process Funnel Analysis:")
    
    process_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
    step_activity = {step: df[df['process_step'] == step]['Client_ID'].nunique() for step in process_steps}
    step_activity_series = pd.Series(step_activity).sort_index()

    start_clients = step_activity.get('start', 0)
    confirm_clients = step_activity.get('confirm', 0)
    
    # Final fix for rate calculation
    if start_clients > 0:
        completion_rate = round((confirm_clients / start_clients * 100), 2)
    else:
        completion_rate = 0

    print("Clients reaching each step of the process:")
    print(step_activity_series.to_frame(name='Unique Clients').to_string())
    print(f"\nOverall Process Completion Rate (Confirm/Start): {completion_rate}%")
    


# --- Main Execution Block (Corrected Function Name) ---
if __name__ == '__main__':
    df_merged = load_and_merge_data(FILE_DEMO, FILE_WEB_1, FILE_WEB_2, FILE_EXPERIMENT)
    
    if df_merged is not None:
        # FIX: Ensure function name is correct: perform_eda_and_cleaning
        df_cleaned = perform_eda_and_cleaning(df_merged) 
        
        if df_cleaned is not None:
            analyze_client_behavior(df_cleaned)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# Silence SettingWithCopyWarning for categorical assignments
pd.options.mode.chained_assignment = None 

# --- Define File Names (Assuming these files are in the same directory) ---
FILE_DEMO = 'df_final_demo.txt'
FILE_WEB_1 = 'df_final_web_data_pt_1.txt'
FILE_WEB_2 = 'df_final_web_data_pt_2.txt'
FILE_EXPERIMENT = 'df_final_experiment_clients.txt'

# --- 1. Data Loading and Merging (Task 1: Data Preparation) ---
def load_and_merge_data(demo_file, web_1_file, web_2_file, experiment_file):
    """Loads and merges all required datasets."""
    print("--- TASK 1: DATA LOADING AND MERGING ---")
    try:
        # Load datasets (using read_csv because your .txt files are CSV-formatted)
        df_demo = pd.read_csv(demo_file)
        df_web_1 = pd.read_csv(web_1_file)
        df_web_2 = pd.read_csv(web_2_file)
        df_experiment = pd.read_csv(experiment_file)

        df_web = pd.concat([df_web_1, df_web_2], ignore_index=True)

        # Standardize client ID column name
        for df_ in [df_experiment, df_demo, df_web]:
            if 'client_id' in df_.columns:
                df_.rename(columns={'client_id': 'Client_ID'}, inplace=True)
        
        # Merge all data
        df_merged = df_experiment.merge(df_demo, on='Client_ID', how='left')
        df_merged = df_merged.merge(df_web, on='Client_ID', how='left')
        print("All datasets merged successfully.")
        return df_merged
    except Exception as e:
        print(f"Error during loading/merging: {e}")
        return None

# --- 1. EDA & Data Cleaning (Task 1: Data Preparation) ---
def perform_eda_and_cleaning(df):
    """Performs initial EDA and cleaning steps."""
    if df is None: return None
    
    print("\n--- TASK 1: EDA AND DATA CLEANING ---")

    # Drop duplicates
    initial_rows = len(df)
    df.drop_duplicates(subset=['Client_ID', 'date_time', 'process_step'], keep='first', inplace=True)

    # Convert date_time and create Date column
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['Date'] = df['date_time'].dt.date
    df['Date'] = pd.to_datetime(df['Date'])

    # Create 'TestGroup' column
    df['TestGroup'] = np.where(df['Variation'] == 'Control', 'Control', 'Test')
    
    print(f"Total rows before cleaning: {initial_rows}")
    print(f"Total rows after cleaning: {len(df)}")
    print(f"Total unique clients after cleaning: {df['Client_ID'].nunique()}")
    
    return df


# --- 2. Client Behavior Analysis (Task 2: Analysis) ---
def analyze_client_behavior(df):
    """Answers questions about client demographics and overall funnel performance."""
    if df is None: return None
    
    print("\n--- TASK 2: CLIENT BEHAVIOR ANALYSIS ---")
    
    # Filter clients with complete demographic data and create a copy
    df_unique_clients = df.drop_duplicates(subset=['Client_ID'])
    df_analysis_clients = df_unique_clients[df_unique_clients['clnt_age'].notnull()].copy() 
    total_clients = df_analysis_clients['Client_ID'].nunique()

    # --- 3.1.1 Gender Analysis (Uses 'gendr') ---
    gender_analysis = df_analysis_clients.groupby('gendr')['Client_ID'].count().sort_values(ascending=False)
    gender_pct = (gender_analysis / total_clients * 100).round(2) 
    print("\n3.1.1 Primary Clients by Gender:")
    print(gender_pct.to_frame(name='Client Pct (%)').to_string()) 

    # --- 3.1.2 Age Group Analysis (Uses 'clnt_age') ---
    age_bins = [18, 30, 45, 60, 100]
    age_labels = ['18-29 (Young)', '30-44 (Middle)', '45-59 (Senior)', '60+ (Retired)']
    df_analysis_clients['Age_Group'] = pd.cut(df_analysis_clients['clnt_age'], bins=age_bins, labels=age_labels, right=False, include_lowest=True)
    
    # Corrected for FutureWarning: Added observed=True
    age_analysis = df_analysis_clients.groupby('Age_Group', observed=True)['Client_ID'].count()
    age_pct = (age_analysis / total_clients * 100).round(2)
    print("\n3.1.2 Primary Clients by Age Group:")
    print(age_pct.to_frame(name='Client Pct (%)').to_string())

    # --- 3.2.1 Tenure Group Analysis (Uses 'clnt_tenure_yr') ---
    tenure_bins = [0, 5, 10, 20, 50]
    tenure_labels = ['<5 (New)', '5-9 (Mid-Term)', '10-19 (Long-Term)', '20+ (Veteran)']
    df_analysis_clients['Tenure_Group'] = pd.cut(df_analysis_clients['clnt_tenure_yr'], bins=tenure_bins, labels=tenure_labels, right=False, include_lowest=True)
    
    # Corrected for FutureWarning: Added observed=True
    tenure_analysis = df_analysis_clients.groupby('Tenure_Group', observed=True)['Client_ID'].count()
    
    # Corrected percentage calculation 
    tenure_pct = tenure_analysis.div(total_clients).mul(100).round(2)
    
    print("\n3.2.1 Primary Clients by Tenure Group (Years Active):")
    print(tenure_pct.to_frame(name='Client Pct (%)').to_string())
    
    # --- 3.3.1 Overall Funnel Performance (Conversion Rate) ---
    print("\n3.3.1 Overall Process Funnel Analysis:")
    
    process_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
    step_activity = {step: df[df['process_step'] == step]['Client_ID'].nunique() for step in process_steps}
    step_activity_series = pd.Series(step_activity).sort_index()

    start_clients = step_activity.get('start', 0)
    confirm_clients = step_activity.get('confirm', 0)
    
    if start_clients > 0:
        completion_rate = round((confirm_clients / start_clients * 100), 2)
    else:
        completion_rate = 0

    print("Clients reaching each step of the process:")
    print(step_activity_series.to_frame(name='Unique Clients').to_string())
    print(f"\nOverall Process Completion Rate (Confirm/Start): {completion_rate}%")
    


# --- Main Execution Block ---
if __name__ == '__main__':
    df_merged = load_and_merge_data(FILE_DEMO, FILE_WEB_1, FILE_WEB_2, FILE_EXPERIMENT)
    
    if df_merged is not None:
        df_cleaned = perform_eda_and_cleaning(df_merged) 
        
        if df_cleaned is not None:
            analyze_client_behavior(df_cleaned)
            
            # --- DISPLAY THE CLEANED TABLE ---
            print("\n--- DISPLAYING FIRST 5 ROWS OF THE CLEANED TABLE (df_cleaned) ---")
            print(df_cleaned.head().to_string())

--- TASK 1: DATA LOADING AND MERGING ---
All datasets merged successfully.

--- TASK 1: EDA AND DATA CLEANING ---
Total rows before cleaning: 449831
Total rows after cleaning: 443896
Total unique clients after cleaning: 70609

--- TASK 2: CLIENT BEHAVIOR ANALYSIS ---

3.1.1 Primary Clients by Gender:
       Client Pct (%)
gendr                
U               34.17
M               33.61
F               32.22
X                0.00

3.1.2 Primary Clients by Age Group:
                Client Pct (%)
Age_Group                     
18-29 (Young)            18.31
30-44 (Middle)           27.03
45-59 (Senior)           31.16
60+ (Retired)            22.98

3.2.1 Primary Clients by Tenure Group (Years Active):
                   Client Pct (%)
Tenure_Group                     
<5 (New)                     7.39
5-9 (Mid-Term)              37.40
10-19 (Long-Term)           39.08
20+ (Veteran)               16.09

3.3.1 Overall Process Funnel Analysis:
Clients reaching each step of the process:
 